In [1]:
import pandas as pd

# Load the dataset
file_path = "../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

# Check class distribution
print(df['sentiment'].value_counts())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [2]:
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing
df['cleaned_review'] = df['review'].apply(preprocess_text)


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [3]:
from sklearn.model_selection import train_test_split

X = df['cleaned_review']
y = df['sentiment'].map({'positive': 1, 'negative': 0})  # Encode labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_features=5000)  # Use top 5000 words

# Fit and transform the training data
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)


In [5]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the model
model = MultinomialNB()

# Train the model
model.fit(X_train_vectors, y_train)


MultinomialNB()

In [6]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test_vectors)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.85

Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      4961
           1       0.85      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [7]:
# Test with custom reviews
new_reviews = ["I absolutely loved this movie!", "It was the worst film I've ever seen."]
new_vectors = vectorizer.transform(new_reviews)

# Predict sentiment
new_predictions = model.predict(new_vectors)

for review, pred in zip(new_reviews, new_predictions):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Review: '{review}' -> Sentiment: {sentiment}")


Review: 'I absolutely loved this movie!' -> Sentiment: Positive
Review: 'It was the worst film I've ever seen.' -> Sentiment: Negative
